<header style="width:100%;position:relative">
  <div style="width:80%;float:right;">
    <h1>False Political Claim Detection</h1>
    <h3>Selección de características y preprocesado</h3>
    <h5>Grupo 2</h5>
  </div>
        <img style="width:15%;" src="images/logo.jpg" alt="UPM" />
</header>

# Índice

1. [Importar librerias](#1.-Importar-librerias)
2. [Variables globales](#2.-Variables-globales)
3. [Carga del dataframe](#3.-Carga-del-dataframe)
4. [Análisis y selección de las características](#4.-Analisis-y-seleccion-de-las-caracteristicas)
5. [Carga de los datos y división en entrenamiento y test](#5.-Carga-de-los-datos-y-division-en-entrenamiento-y-test)
6. [Preprocesado de los datos](#6.-Preprocesado-de-los-datos)
    * 6.1 [Introducción](#6.1-Introduccion)
    * 6.2 [Casos de preprocesado](#6.2-Casos-de-preprocesado)
    * 6.3 [Conclusiones](#6.3-Conclusiones)
7. [Referencias](#7.-Referencias)


## 1. Importar librerias

In [1]:
# General import and load data
import pandas as pd
import numpy as np

# Preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# Splitting
from sklearn.model_selection import train_test_split

# Estimators
from sklearn.ensemble import RandomForestClassifier

# Evaluation
from sklearn.metrics import accuracy_score, f1_score

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch

print("Todas las librerias fueron correctamente importadas.")

C:\Users\alexf\Downloads\Reto2_ML-ABID\mi_entorno\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\alexf\Downloads\Reto2_ML-ABID\mi_ent

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
No module named 'torch.distributed.tensor'

## 2. Variables globales y funciones auxiliares

Se fija un *seed* para todo el documento para fijar la aleatoriedad y así obtener resultados replicables.

In [ ]:
seed = 42

Función utilizada para seleccionar el *statement* con el que se quiere entrenar el modelo.

In [ ]:
def selStatement(case, df):
    mapping = {
        1: 'statement',
        2: 'speaker_statement',
        3: 'speaker_statement_subject',
        4: 'speaker_party_statement',
        5: 'speaker_party_statement_subject',
        6: 'speaker_speaker-job_party_statement',
        7: 'speaker_speaker-job_party_statement_subject'
    }

    if case not in mapping:
        raise ValueError("El valor de 'case' no es válido. Debe estar entre 1 y 7.")

    return df[mapping[case]]

Función utilizada para seleccionar el modelo con el que se quiere entrenar.

In [ ]:
def selModel(model):
    mapping = {
        'deberta': 'microsoft/deberta-v3-base',
        'roberta': 'distilbert/distilroberta-base',
        'destilado': 'distilbert/distilbert-base-uncased',
        'berta': 'google-bert/bert-base-uncased'
    }

    if model not in mapping:
        raise ValueError("El modelo seleccionado no existe entre los disponibles. Revise el valor introducido.")

    return mapping[model]

Función encargada de evaluar el modelo de *Hugging Face*.

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions),
    }

Función para transformar el *dataframe* en *dataset*.

In [ ]:
class OwnDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) 
                for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

## 3. Carga del dataframe

Se cargan los datos de la ruta *formated/train_hugging.csv*, los cuales son los datos ya procesados por uno de nuestros compañeros.

In [ ]:
url = "train_hugging.csv"
df = pd.read_csv(url)

print("Datos cargados correctamente\n")

También cargamos el test que debemos predecir para Kaggle de la ruta *formated/test_hugging.csv*.

In [ ]:
url = "test_hugging.csv"
df_test = pd.read_csv(url)

print("Test cargado correctamente\n")

## 4. Analisis de las caracteristicas

Se configura pandas para que muestre todas las columnas y después se realiza un head para ver el contenido de las mismas.

In [ ]:
pd.set_option('display.max_columns', None)
df.head()

Se muestran todas las columnas para poder observarlas.

In [ ]:
df.columns

Se muestran todas las columnas en formato código para que sea más sencillo su uso. Además se estructuran en diversas categorias.

In [ ]:
all_features = [
    # Identificador
    'id',

    # Etiqueta objetivo
    'label',

    # Texto original
    'statement',

    # Texto original + Speaker
    'speaker_statement',

    # Texto original + Speaker + Subject
    'speaker_statement_subject',

    # Texto original + Speaker + Party affilation
    'speaker_party_statement',
    
    # Texto original + Speaker + Party affilation + Subject
    'speaker_party_statement_subject',

    # Texto original + Speaker + Party affilation + Speaker Job
    'speaker_speaker-job_party_statement',

    # Texto original + Speaker + Party affilation + Speaker Job + Subject
    'speaker_speaker-job_party_statement_subject'
]

Como se observa se ha buscado unificar de forma progresiva las columnas. Además, se ha intentado no quitar palabras del texto original ya que los modelos de *Hugging Face* ya los tratan internamente mediante su propio *tokenizador*. Cada una de las columnas representara un caso de entrenamiento. A continuación se muestra un ejemplo de cada una de las columnas:

* **Caso 1 -** ***statement***

In [ ]:
df['statement'].iloc[0]

* **Caso 2 -** ***speaker_statement***

In [ ]:
df['speaker_statement'].iloc[0]

* **Caso 3 -** ***speaker_statement_subject***

In [ ]:
df['speaker_statement_subject'].iloc[0]

* **Caso 4 -** ***speaker_party_statement***

In [ ]:
df['speaker_party_statement'].iloc[0]

* **Caso 5 -** ***speaker_party_statement_subject***

In [ ]:
df['speaker_party_statement_subject'].iloc[0]

* **Caso 6 -** ***speaker_speaker-job_party_statement***

In [ ]:
df['speaker_speaker-job_party_statement'].iloc[0]

* **Caso 7 -** ***speaker_speaker-job_party_statement_subject***

In [ ]:
df['speaker_speaker-job_party_statement_subject'].iloc[0]

## 6. Entrenamiento

El objetivo de este notebook es buscar mejorar nuestro clasificador utilizando solamente la columna textual. Para ello se ha realizado el preprocesamiento explicado en el anterior apartado. A la hora de entrenarlos se utilizarán modelos famosos de *Hugging Face*.

Como puede observarse en el apartado dos, se han creado dos funciones para centralizar tanto el caso de entrenamiento como la selección del modelo.

### 6.1. Caso 1

#### 6.1.1 Carga de los datos y division en entrenamiento y test

Cargamos las caracteristicas seleccionadas en la variable X y el objetivo *label* en la variable y. Para la seleccion de la variable X se utilizará la función auxiliar *selStatement*, introducciendo como parámetros el caso de entrenamiento seleccionado y el *dataframe* donde estan almacenados los datos.

In [ ]:
X =  selStatement(1, df)
y = df['label']

Se realiza un *iloc* para comprobar de forma sencilla que hemos escogido el caso de entrenamiento correcto

In [ ]:
X.iloc[0]

Se separa en entrenamiento y test.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

#### 6.1.2 Entrenamiento

Seleccionamos el modelo mediante la función *selModel* y el parámetro *'deberta'*.

In [ ]:
model_name = selModel('roberta')
print("Modelo seleccionado:", model_name)

Seleccionamos el tokenizador del modelo.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Tokenizamos los textos.

In [ ]:
train_encodings = tokenizer(list(X), truncation=True, padding=True)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True)

Transformamos los *dataframes* en *dataset* con la funcion propia *OwnDataset*.

In [ ]:
train_dataset = OwnDataset(train_encodings, list(y_train))
test_dataset = OwnDataset(test_encodings, list(y_test))

Cargamos el modelo.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results_caso1",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True
)

Definimos un entrenador.

In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

Entrenamos el modelo.

In [ ]:
trainer.train()

Evaluamos los estadísticos establecidos previamente.

In [ ]:
trainer.evaluate()

Almacenamos el modelo para poder reutilizarse.

In [ ]:
trainer.save_model("roberta_caso1")
tokenizer.save_pretrained("roberta_caso1")

### 6.2. Caso 2

#### 6.2.1 Carga de los datos y division en entrenamiento y test

Cargamos las caracteristicas seleccionadas en la variable X y el objetivo *label* en la variable y. Para la seleccion de la variable X se utilizará la función auxiliar *selStatement*, introducciendo como parámetros el caso de entrenamiento seleccionado y el *dataframe* donde estan almacenados los datos.

In [ ]:
X =  selStatement(2, df)
y = df['label']

Se realiza un *iloc* para comprobar de forma sencilla que hemos escogido el caso de entrenamiento correcto

In [ ]:
X.iloc[0]

Se separa en entrenamiento y test.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

#### 6.2.2 Entrenamiento

Seleccionamos el modelo mediante la función *selModel* y el parámetro *'deberta'*.

In [ ]:
model_name = selModel('roberta')
print("Modelo seleccionado:", model_name)

Seleccionamos el tokenizador del modelo.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Tokenizamos los textos.

In [ ]:
train_encodings = tokenizer(list(X), truncation=True, padding=True)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True)

Transformamos los *dataframes* en *dataset* con la funcion propia *OwnDataset*.

In [ ]:
train_dataset = OwnDataset(train_encodings, list(y_train))
test_dataset = OwnDataset(test_encodings, list(y_test))

Cargamos el modelo.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results_caso2",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True
)

Definimos un entrenador.

In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

Entrenamos el modelo.

In [ ]:
trainer.train()

Evaluamos los estadísticos establecidos previamente.

In [ ]:
trainer.evaluate()

Almacenamos el modelo para poder reutilizarse.

In [ ]:
trainer.save_model("roberta_caso2")
tokenizer.save_pretrained("roberta_caso2")

### 6.4. Caso 3

#### 6.4.1 Carga de los datos y division en entrenamiento y test

Cargamos las caracteristicas seleccionadas en la variable X y el objetivo *label* en la variable y. Para la seleccion de la variable X se utilizará la función auxiliar *selStatement*, introducciendo como parámetros el caso de entrenamiento seleccionado y el *dataframe* donde estan almacenados los datos.

In [ ]:
X =  selStatement(3, df)
y = df['label']

Se realiza un *iloc* para comprobar de forma sencilla que hemos escogido el caso de entrenamiento correcto

In [ ]:
X.iloc[0]

Se separa en entrenamiento y test.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

#### 6.4.2 Entrenamiento

Seleccionamos el modelo mediante la función *selModel* y el parámetro *'deberta'*.

In [ ]:
model_name = selModel('roberta')
print("Modelo seleccionado:", model_name)

Seleccionamos el tokenizador del modelo.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Tokenizamos los textos.

In [ ]:
train_encodings = tokenizer(list(X), truncation=True, padding=True)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True)

Transformamos los *dataframes* en *dataset* con la funcion propia *OwnDataset*.

In [ ]:
train_dataset = OwnDataset(train_encodings, list(y_train))
test_dataset = OwnDataset(test_encodings, list(y_test))

Cargamos el modelo.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results_caso3",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True
)

Definimos un entrenador.

In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

Entrenamos el modelo.

In [ ]:
trainer.train()

Evaluamos los estadísticos establecidos previamente.

In [ ]:
trainer.evaluate()

Almacenamos el modelo para poder reutilizarse.

In [ ]:
trainer.save_model("roberta_caso3")
tokenizer.save_pretrained("roberta_caso3")

### 6.5. Caso 4

#### 6.6.1 Carga de los datos y division en entrenamiento y test

Cargamos las caracteristicas seleccionadas en la variable X y el objetivo *label* en la variable y. Para la seleccion de la variable X se utilizará la función auxiliar *selStatement*, introducciendo como parámetros el caso de entrenamiento seleccionado y el *dataframe* donde estan almacenados los datos.

In [ ]:
X =  selStatement(4, df)
y = df['label']

Se realiza un *iloc* para comprobar de forma sencilla que hemos escogido el caso de entrenamiento correcto

In [ ]:
X.iloc[0]

Se separa en entrenamiento y test.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

#### 6.5.2 Entrenamiento

Seleccionamos el modelo mediante la función *selModel* y el parámetro *'deberta'*.

In [ ]:
model_name = selModel('roberta')
print("Modelo seleccionado:", model_name)

Seleccionamos el tokenizador del modelo.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Tokenizamos los textos.

In [ ]:
train_encodings = tokenizer(list(X), truncation=True, padding=True)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True)

Transformamos los *dataframes* en *dataset* con la funcion propia *OwnDataset*.

In [ ]:
train_dataset = OwnDataset(train_encodings, list(y_train))
test_dataset = OwnDataset(test_encodings, list(y_test))

Cargamos el modelo.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results_caso4",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True
)

Definimos un entrenador.

In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

Entrenamos el modelo.

In [ ]:
trainer.train()

Evaluamos los estadísticos establecidos previamente.

In [ ]:
trainer.evaluate()

Almacenamos el modelo para poder reutilizarse.

In [ ]:
trainer.save_model("roberta_caso4")
tokenizer.save_pretrained("roberta_caso4")

### 6.6. Caso 5

#### 6.6.1 Carga de los datos y division en entrenamiento y test

Cargamos las caracteristicas seleccionadas en la variable X y el objetivo *label* en la variable y. Para la seleccion de la variable X se utilizará la función auxiliar *selStatement*, introducciendo como parámetros el caso de entrenamiento seleccionado y el *dataframe* donde estan almacenados los datos.

In [ ]:
X =  selStatement(5, df)
y = df['label']

Se realiza un *iloc* para comprobar de forma sencilla que hemos escogido el caso de entrenamiento correcto

In [ ]:
X.iloc[0]

Se separa en entrenamiento y test.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

#### 6.6.2 Entrenamiento

Seleccionamos el modelo mediante la función *selModel* y el parámetro *'deberta'*.

In [ ]:
model_name = selModel('roberta')
print("Modelo seleccionado:", model_name)

Seleccionamos el tokenizador del modelo.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Tokenizamos los textos.

In [ ]:
train_encodings = tokenizer(list(X), truncation=True, padding=True)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True)

Transformamos los *dataframes* en *dataset* con la funcion propia *OwnDataset*.

In [ ]:
train_dataset = OwnDataset(train_encodings, list(y_train))
test_dataset = OwnDataset(test_encodings, list(y_test))

Cargamos el modelo.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results_caso5",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True
)

Definimos un entrenador.

In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

Entrenamos el modelo.

In [ ]:
trainer.train()

Evaluamos los estadísticos establecidos previamente.

In [ ]:
trainer.evaluate()

Almacenamos el modelo para poder reutilizarse.

In [ ]:
trainer.save_model("roberta_caso5")
tokenizer.save_pretrained("roberta_caso5")

### 6.7. Caso 6

#### 6.7.1 Carga de los datos y division en entrenamiento y test

Cargamos las caracteristicas seleccionadas en la variable X y el objetivo *label* en la variable y. Para la seleccion de la variable X se utilizará la función auxiliar *selStatement*, introducciendo como parámetros el caso de entrenamiento seleccionado y el *dataframe* donde estan almacenados los datos.

In [12]:
X =  selStatement(6, df)
y = df['label']

NameError: name 'selStatement' is not defined

Se realiza un *iloc* para comprobar de forma sencilla que hemos escogido el caso de entrenamiento correcto

In [ ]:
X.iloc[0]

Se separa en entrenamiento y test.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

#### 6.7.2 Entrenamiento

Seleccionamos el modelo mediante la función *selModel* y el parámetro *'deberta'*.

In [ ]:
model_name = selModel('roberta')
print("Modelo seleccionado:", model_name)

Seleccionamos el tokenizador del modelo.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Tokenizamos los textos.

In [ ]:
train_encodings = tokenizer(list(X), truncation=True, padding=True)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True)

Transformamos los *dataframes* en *dataset* con la funcion propia *OwnDataset*.

In [ ]:
train_dataset = OwnDataset(train_encodings, list(y_train))
test_dataset = OwnDataset(test_encodings, list(y_test))

Cargamos el modelo.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results_caso6",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True
)

Definimos un entrenador.

In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

Entrenamos el modelo.

In [ ]:
trainer.train()

Evaluamos los estadísticos establecidos previamente.

In [ ]:
trainer.evaluate()

Almacenamos el modelo para poder reutilizarse.

In [ ]:
trainer.save_model("roberta_caso6")
tokenizer.save_pretrained("roberta_caso6")

### 6.8. Caso 7

#### 6.8.1 Carga de los datos y division en entrenamiento y test

Cargamos las caracteristicas seleccionadas en la variable X y el objetivo *label* en la variable y. Para la seleccion de la variable X se utilizará la función auxiliar *selStatement*, introducciendo como parámetros el caso de entrenamiento seleccionado y el *dataframe* donde estan almacenados los datos.

In [ ]:
X =  selStatement(7, df)
y = df['label']

Se realiza un *iloc* para comprobar de forma sencilla que hemos escogido el caso de entrenamiento correcto

In [ ]:
X.iloc[0]

Se separa en entrenamiento y test.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

#### 6.8.2 Entrenamiento

Seleccionamos el modelo mediante la función *selModel* y el parámetro *'deberta'*.

In [ ]:
model_name = selModel('roberta')
print("Modelo seleccionado:", model_name)

Seleccionamos el tokenizador del modelo.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Tokenizamos los textos.

In [ ]:
train_encodings = tokenizer(list(X), truncation=True, padding=True)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True)

Transformamos los *dataframes* en *dataset* con la funcion propia *OwnDataset*.

In [ ]:
train_dataset = OwnDataset(train_encodings, list(y_train))
test_dataset = OwnDataset(test_encodings, list(y_test))

Cargamos el modelo.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results_caso7",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True
)

Definimos un entrenador.

In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

Entrenamos el modelo.

In [ ]:
trainer.train()

Evaluamos los estadísticos establecidos previamente.

In [ ]:
trainer.evaluate()

Almacenamos el modelo para poder reutilizarse.

In [ ]:
trainer.save_model("roberta_caso7")
tokenizer.save_pretrained("roberta_caso7")

# 7. Referencias

* [ColumnTransformer. (s. f.). Scikit-learn.](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html)
* [RandomForestClassifier. (s. f.). Scikit-learn.](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
* [RandomUnderSampler — Version 0.13.0. (s. f.). Imbalance learn.](https://imbalanced-learn.org/stable/references/generated/imblearn.under_sampling.RandomUnderSampler.html)
* [SMOTE — Version 0.13.0. (s. f.). Imbalanced learn.](https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html)
* [StandardScaler. (s. f.). Scikit-learn.](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)
* [TfidfVectorizer. (s. f.). Scikit-learn.](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)
* [train_test_split. (s. f.). Scikit-learn.](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)
* [TruncatedSVD. (s. f.). Scikit-learn.](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html)